OData API

https://oda.ft.dk/Home/OdataQuery

https://www.odata.org/

https://www.ft.dk/-/media/sites/ft/pdf/dokumenter/aabne-data/oda-browser_brugervejledning.ashx

https://www.odata.org/documentation/odata-version-3-0/url-conventions/

https://www.odata.org/documentation/odata-version-3-0/odata-version-3-0-core-protocol/

Source root Link: "https://oda.ft.dk/api/"

ftp://oda.ft.dk/ODAXML/Referat/samling/


## Folketinget data

The data related to the meetings in Folketinget is collected following the guide on Folketinget's homepage:
https://www.ft.dk/-/media/sites/ft/pdf/dokumenter/aabne-data/oda-browser_brugervejledning.ashx.
Transcripts from all meetings going back to 2009 are available for download in xml format using a login described in the guide and some specific ftp software. Because it is interesting to compare topics from Twitter with topics in Folketinget, only the transcripts from the same period as the tweets are used for further analysis. 71% of the available tweets are posted in 2018 and after.

The years in Folketinget do not follow the calender year, as they start and end the first Tuesday of October. So, the meetings from 2018 are actually from October 2017 to October 2018. The data from 2017 are kept for the purpose of having slightly more data and because it is assumed that topics vary little enought for it to still be relevant.

In total 284 meeting stranscripts amounting to 256.6Mb of data has been downloaded and parsed for analysis.

It is not stated how the transcripts are made, however, they are proofread before publication. 


In [ ]:
#pip install bs4 lxml

In [10]:
import requests
import urllib.request
import os
import io
import json

import numpy as np
import pandas as pd
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer

from bs4 import BeautifulSoup as bs
from bs4.element import Comment
import lxml

import xml.etree.ElementTree as et 
from xml.dom import minidom

import re

import dateutil
import datetime as dt
from dateutil.easter import *
from dateutil.rrule import *
from dateutil.parser import *
from datetime import *

In [11]:
path = os.getcwd()

In [12]:
path = path+'/samling/'

In [13]:
path

'/Users/frederikkromannhansen/Documents/GitHub/politiker/samling/'

In [8]:
ls samling/20181

20181_M10_helemoedet.xml       20181_M52_helemoedet.xml
20181_M11_helemoedet.xml       20181_M53_helemoedet.xml
20181_M12_helemoedet.xml       20181_M54_helemoedet.xml
20181_M13_helemoedet.xml       20181_M55_helemoedet.xml
20181_M14_helemoedet.xml       20181_M56_helemoedet.xml
20181_M15_helemoedet.xml       20181_M57_helemoedet.xml
20181_M16_helemoedet.xml       20181_M58_helemoedet.xml
20181_M17_helemoedet.xml       20181_M59_helemoedet.xml
20181_M18_helemoedet.xml       20181_M5_helemoedet.xml
20181_M19_helemoedet.xml       20181_M60_helemoedet.xml
20181_M1_helemoedet.xml        20181_M61_helemoedet.xml
20181_M1_helemoedet_clean.xml  20181_M62_helemoedet.xml
20181_M20_helemoedet.xml       20181_M63_helemoedet.xml
20181_M21_helemoedet.xml       20181_M64_helemoedet.xml
20181_M22_helemoedet.xml       20181_M65_helemoedet.xml
20181_M23_helemoedet.xml       20181_M66_helemoedet.xml
20181_M24_helemoedet.xml       20181_M67_helemoedet.xml

20181_M26_helemoedet.xml       20181_M69_helemoe

# Parsing XML

In [15]:
for subdir, dir, files in os.walk(path):
    for file in files:
        filepath = subdir+'/' + file
        if filepath.endswith(".xml"):
            xmlTree = et.parse(filepath)
            
elemList = []

for elem in xmlTree.iter():
    elemList.append(elem.tag)

# now I remove duplicities - by convertion to set and back to list
#elemList = list(set(elemList))

# Just printing out the result
from collections import Counter
Counter(elemList)

Counter({'Dokument': 1,
         'MetaMeeting': 1,
         'ParliamentarySession': 1,
         'ParliamentaryGroup': 1,
         'MeetingNumber': 1,
         'DateOfSitting': 1,
         'Location': 1,
         'EdixiDocLocation': 1,
         'AudioFileFolder': 1,
         'TitelGruppe': 1,
         'Titel': 1,
         'Linea': 126,
         'Char': 138,
         'UnderTitel': 1,
         'DagsordenPlan': 1,
         'Rubrica': 10,
         'PunktTekst': 8,
         'Exitus': 72,
         'DagsordenPunkt': 5,
         'MetaFTAgendaItem': 5,
         'ItemNo': 5,
         'FTCase': 5,
         'FTCaseNumber': 5,
         'FTCaseType': 5,
         'FTCaseStage': 5,
         'ShortTitle': 5,
         'Aktivitet': 10,
         'Tale': 12,
         'Taler': 12,
         'MetaSpeakerMP': 12,
         'OratorFirstName': 12,
         'OratorLastName': 12,
         'GroupNameShort': 12,
         'OratorRole': 12,
         'TalerTitel': 12,
         'TaleSegment': 12,
         'MetaSpeechSegme

In [28]:
for speech in root.iter('Tale'):
    tags=[node.tag for node in speech.iter()]

In [29]:
tags

['Tale',
 'Taler',
 'MetaSpeakerMP',
 'OratorFirstName',
 'OratorLastName',
 'GroupNameShort',
 'OratorRole',
 'TalerTitel',
 'Linea',
 'Char',
 'Char',
 'TaleSegment',
 'MetaSpeechSegment',
 'LastModified',
 'EdixiStatus',
 'StartDateTime',
 'TekstGruppe',
 'Exitus',
 'Linea',
 'Char',
 'Linea',
 'Char',
 'Linea',
 'Char']

In [20]:
df.head()

,MeetingID,ItemNo,StartDateTime,OratorFirstName,OratorLastName,GroupNameShort,OratorRole,TekstGruppe
0,66,0,2019-02-27T13:00:08,Pia,Kjærsgaard,DF,formand,Mødet er åbnet.
1,66,0,2019-02-27T13:01:34,Pia,Kjærsgaard,DF,formand,Det første spørgsmål er til justitsministeren...
2,66,0,2019-02-27T13:01:43,Pia,Kjærsgaard,DF,formand,Værsgo for at oplæse spørgsmålet.
3,66,0,2019-02-27T13:01:45,Christian,Langballe,DF,medlem,Spørgsmålet lyder: Vil regeringen stå fast på...
4,66,0,2019-02-27T13:02:05,Pia,Kjærsgaard,DF,formand,"Værsgo, ministeren."


In [32]:
print(df.TekstGruppe[74023])

 Til formand har Socialdemokratiet, Dansk Folkeparti, Venstre, Liberal Alliance, Det Konservative Folkeparti, Nunatta Qitornai, Tjóðveldi og Javnaðarflokkurin indstillet fru Pia Kjærsgaard. Der foreligger ikke andre indstillinger.


In [38]:
starttid_vec = []
sluttid_vec=[]
navn_vec = []
efternavn_vec = []
tekst_vec = []
parti_vec = []
rolle_vec = []
meetingID_vec = []


# Go through all the folders and files containing transcripts
for subdir, dir, files in os.walk(path):
    for file in files:
        filepath = subdir+'/' + file
        if (filepath.endswith(".xml")) and not (filepath.endswith("clean.xml")):
            tree = et.parse(filepath)
            root = tree.getroot() 

            for meeting in root.iter('MeetingNumber'):
                meetingID = meeting.text
            # Do this if the speaker has an associated party, i.e. not a minister or the chairman
            for speech in root.iter('Tale'):

                tags=[node.tag for node in speech.iter()]
                if('TekstGruppe' in tags):
                    if('GroupNameShort' in tags):
                        for node in speech.iter():
                            if(node.tag == 'StartDateTime'):
                                starttid_vec.append(node.text)
                            elif(node.tag == 'OratorFirstName'):
                                navn_vec.append(node.text)
                            elif(node.tag == 'OratorLastName'):
                                efternavn_vec.append(node.text)
                            elif(node.tag == 'OratorRole'):
                                rolle_vec.append(node.text)
                            elif(node.tag=='TekstGruppe'):
                                # This tag is preculiar, as there are multiple children ending with 'Char' as the youngest.
                                # Concate all text within 'TekstGruppe'.
                                tekstgruppe = ''
                                for k in node.iter():
                                    if k.tag=="Char":
                                        tekstgruppe+=' '+k.text
                                tekst_vec.append(tekstgruppe)
                            elif(node.tag=='GroupNameShort'):
                                parti_vec.append(node.text)
                                
                                meetingID_vec.append(meetingID)


                            # If there are more text than speaker names, remove the excess.
                            # We will loose a few pieces of text but only around 1%
                            if len(tekst_vec)>len(navn_vec):
                                tekst_vec.pop()
                            if len(starttid_vec)>len(navn_vec):
                                starttid_vec.pop()

                    # Do this if the speaker is either a minister or the chairman
                    else:
                        parti_vec.append('Ukendt')
                        for node in speech.iter():
                            if(node.tag == 'StartDateTime'):
                                starttid_vec.append(node.text)
                            elif(node.tag == 'OratorFirstName'):
                                navn_vec.append(node.text)
                            elif(node.tag == 'OratorLastName'):
                                efternavn_vec.append(node.text)
                            elif(node.tag == 'OratorRole'):
                                rolle_vec.append(node.text)
                            elif(node.tag=='TekstGruppe'):
                                # This tag is preculiar, as there are multiple children ending with 'Char' as the youngest.
                                # Concate all text within 'TekstGruppe'.
                                tekstgruppe = ''
                                for k in node.iter():
                                    if k.tag=="Char":
                                        tekstgruppe+=' '+k.text
                                tekst_vec.append(tekstgruppe)
                                
                                meetingID_vec.append(meetingID)

                            # If there are more text than speaker names, remove the excess.
                            # We will loose a few pieces of text but only around 1%
                            if len(tekst_vec)>len(navn_vec):
                                tekst_vec.pop()
                            if len(starttid_vec)>len(navn_vec):
                                starttid_vec.pop()
                    


# Add everuthing to a dictionary and transform it into a Pandas dataframe
dictionary= {'MeetingID':meetingID_vec, 'StartDateTime':starttid_vec,'OratorFirstName':navn_vec,'OratorLastName':efternavn_vec,
                 'GroupNameShort':parti_vec,'OratorRole':rolle_vec ,'TekstGruppe':tekst_vec} # 'EndDateTime':sluttid_vec,
df = pd.DataFrame(dictionary)

print('In total, there are {} lines of text in the meetings.'.format(len(df)))
#df.to_csv(path[0:26]+'csv')

In total, there are 110438 lines of text in the meetings.


In [39]:
len(meetingID_vec)

110438

In [40]:
### Cleaning the data ###

# The chariman (formand) does not add value to the context as he or she only moderates the debate
# by passing on the word from person to person.
# Also some meta data is contained regarding when the meeting ended (MødeSlut). This is also irrelevant.
df=df[df['OratorRole']!='formand']
df=df[df['OratorRole']!='MødeSlut']
df['OratorRole'].dropna(axis=0,inplace=True)
df.reset_index(drop=True,inplace=True)

# Generating a column of the full name of the speakers
df['FullName'] = df['OratorFirstName']+' '+df['OratorLastName']

# Making start time into datetime64 format
df['StartDateTime'] = df['StartDateTime'].apply(lambda x: dateutil.parser.parse(x))

df['MeetingID']=df['MeetingID'].apply(lambda x: int(x))
#df['ItemNo']=df['ItemNo'].apply(lambda x: int(x))

In [ ]:
df_agg_party_date = df.groupby(['GroupNameShort', df['StartDateTime'].dt.year, df['StartDateTime'].dt.month, df['StartDateTime'].dt.day])['TekstGruppe'].apply(lambda x : ' '.join(x))

In [ ]:
df_agg_party_date.sort_index().head()

In [ ]:
df_agg_name = df.groupby(['FullName'])['TekstGruppe'].apply(lambda x : ' '.join(x))
pd.DataFrame(df_agg_name).head()

In [ ]:
df_agg_date_party = df.groupby([df['StartDateTime'].dt.year, df['StartDateTime'].dt.month, df['StartDateTime'].dt.day, 'GroupNameShort'])['TekstGruppe'].apply(lambda x : ' '.join(x)).sort_index()
pd.DataFrame(df_agg_date_party).head()

In [ ]:
df['MeetingID']=df['MeetingID'].apply(lambda x: int(x))

In [ ]:
df.info()

In [41]:
df_agg_date = df.groupby([df['StartDateTime'].dt.year, df['StartDateTime'].dt.month, df['StartDateTime'].dt.day])['TekstGruppe'].apply(lambda x : ' '.join(x)).sort_index()
pd.DataFrame(df_agg_date).head()

TekstGruppe
StartDateTime StartDateTime StartDateTime                                                   
2018          10            2               Deres Majestæt, Deres Kongelige Højheder. I e...
                            4               Tid er en underlig størrelse. Jeg er f.eks. s...
                            9               Tak for det, formand. Nu får jeg anledning ti...
                            10              Tak for det. Lovforslaget omfatter tre område...
                            11              Tak, hr. formand. Det her er to forslag, der ...

In [ ]:
df_agg_date_name = df.groupby([df['StartDateTime'].dt.year, df['StartDateTime'].dt.month, df['StartDateTime'].dt.day, 'FullName'])['TekstGruppe'].apply(lambda x : ' '.join(x)).sort_index()
pd.DataFrame(df_agg_date_name).head()

In [ ]:
df = df[df['FullName'].notnull()]

In [ ]:
df.head()

In [ ]:
df.reset_index(inplace=True, drop=True)

In [ ]:
df_agg_date[1]

## Topics

In [ ]:
stopwords = open('stopord.txt').read().splitlines() #Link: https://gist.github.com/berteltorp/0cf8a0c7afea7f25ed754f24cfc2467b

In [ ]:
# convert the text to a tf-idf weighted term-document matrix
from __future__ import division

# Hyperparameters
corpus = 700000 #Der er ca. 66247 forskellige ord, efter stopord
l1_ratio = 0
max_iter = 50000
no_components = 20 #number of topics

# The words should appear at least 'min_df' times and at most 'max_df' pr document. Decimals indicate proportions
min_df = 2
max_df = 0.6

data=df_agg_date_name

vectorizer = TfidfVectorizer(max_features=corpus, min_df=min_df, max_df=max_df, stop_words=stopwords)

X = vectorizer.fit_transform(data)
 
idx_to_word = np.array(vectorizer.get_feature_names())


# apply Non-negative Matrix Factorization to get topics using TF-IDF
nmf = NMF(n_components=no_components, solver="mu", l1_ratio = l1_ratio, max_iter = max_iter, random_state=1)

W = nmf.fit_transform(X)

H = nmf.components_
 
# print the topics
 
for i, topic in enumerate(H):
 
    print("Topic {}: {}".format(i + 1, ", ".join([str(x) for x in idx_to_word[topic.argsort()[-10:]]])))
    

In [ ]:
# List of words
idx_to_word

In [ ]:
# Last topic's most defining words
print(topic.argsort()[-10:])
print(idx_to_word[topic.argsort()[-10:]])

In [ ]:
#for i in range(len(W)):
#    print(W[i])

In [ ]:
#for i in range(len(H)):
#    print(H[i])

In [ ]:
print('There are ', len(data), ' rows in this aggregation')
print('and from these, ', W.shape[1], ' topics are computed.')
print()
print('There are ', len(idx_to_word),' words in the corpus.')
print(H.shape)

In [ ]:
idx_to_word[topic.argsort()[-10:]]

In [ ]:
pd.concat([data, W], axis=0, join='outer', ignore_index=False, keys=None,
          levels=None, names=None, verify_integrity=False, copy=True)

In [ ]:
pd.set_option('display.float_format', lambda x: '%.8f' % x)
#pd.reset_option('display.float_format')
pd.DataFrame(W)

In [ ]:
pd.DataFrame(H)

In [ ]:
# Write to a .csv file

#with open('topics_and_weights.csv', 'w', newline='', encoding='utf-16') as csvfile:
 #   topicwriter = csv.writer(csvfile, delimiter=',',
  #                          quotechar='"', quoting=csv.QUOTE_MINIMAL)
# Create the header
   # topicwriter.writerow(['Topic','Word','Weight'])

In [ ]:
# print the topics for tableau
 
#for i, topic in enumerate(H):
 
 #   max_weights = topic.argsort()[-10:]
  #  words = [str(x) for x in idx_to_word[topic.argsort()[-10:]]]
     
   # for count, item in enumerate(max_weights):
        
    #    with open('topics_and_weights.csv', 'a', newline='', encoding='utf-16') as csvfile:
     #       topicwriter = csv.writer(csvfile, delimiter=',',
      #                      quotechar='"', quoting=csv.QUOTE_MINIMAL)
       #     topicwriter.writerow(["Topic"+str(i+1) ,words[count],H[i,item]])

In [ ]:
#stopwords_edit = open('stopord.txt','a')
#stopwords_edit.write('\nenhedslisten')
#stopwords_edit.close()